In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, NavigableString

URL = "https://www.stuttgart.de/leichte-sprache-index"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [4]:
category_elements = soup.find_all("section", class_="SP-Section")

for category_element in category_elements:
    print(category_element.prettify(), end="\n"*2)

<section aria-labelledby="abschriften-urkunden-ausweise-paesse" class="SP-Section">
 <h2 class="SP-Headline--section" id="abschriften-urkunden-ausweise-paesse">
  Abschriften &amp; Urkunden, Ausweise &amp; Pässe
 </h2>
 <div class="SP-Section__content">
  <div class="SP-LinkList">
   <ul class="SP-LinkList__list">
    <li class="SP-LinkList__item">
     <a class="SP-Link SP-Iconized--left" href="/organigramm/leistungen/personalausweis-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy">
      <svg aria-hidden="true" class="SPi SPi-link SP-Link__icon SP-Iconized__icon" focusable="false">
       <use href="#SPi-link">
       </use>
      </svg>
      <!--googleoff: index-->
      <span class="SP-Link__text SP-Iconized__text">
       <span class="SP-Link__title">
        Personal-Ausweis beantragen (Leichte Sprache)
       </span>
      </span>
      <!--googleon: index-->
     </a>
    </li>
    <li class="SP-LinkList__item">
     <a class="SP-Link SP-Iconized--left" href="/organigr

In [5]:
links = []
easyLink = "?sp%3Aout=easy"
for category_element in category_elements:
    link_elements = category_element.find_all("a", class_="SP-Link SP-Iconized--left")
    for link_element in link_elements:
        if easyLink in link_element["href"]:
            links.append("https://stuttgart.de"+link_element["href"])
        
        
print(links)

['https://stuttgart.de/organigramm/leistungen/personalausweis-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/personalausweis-wegen-namenaenderung-beantragen-heirat-oder-scheidung-.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/reisepass-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/reisepass-beantragen-vorlaeufig.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/schwerbehinderten-ausweis-beantragen.php?sp%3Aout=easy', 'https://stuttgart.de/organigramm/leistungen/urkunden-und-beglaubigte-abschriften-geburt-heirat-lebens-partnerschaft-und-sterbefall.php?sp%3Aout=easy', 'https://stuttgart.de/buergerinnen-und-buerger/menschen-mit-behinderung/die-beauftragte-fuer-menschen-mit-behinderung.php?sp%3Aout=easy', 'https://stuttgart.de/service/behoerdennummer-115/index.php?sp%3Aout=easy', 'https://stuttgart.de/buergerinnen-und-buerger/menschen-mit-behind

In [21]:
df = pd.DataFrame(columns=['URL', 'Titel', 'Text_Leicht', 'Text_Allgemein'])

In [22]:
i = 1
for url in links:
    try:
        url2 = url.replace("?sp%3Aout=easy", "")
        page = requests.get(url)
        page2 = requests.get(url2)

        soup = BeautifulSoup(page.content, "html.parser")
        soup2 = BeautifulSoup(page2.content, "html.parser")

        result = soup.find("h1", class_="SP-Headline--article")
        title = result.text

        print(title, url)

        result = soup.find("div", class_="SP-ArticleContent")
        for div in result.find_all("section", class_="SP-Text"):
            for h2 in div.find_all("h2", id="kontakt"):
                div.decompose()

        li_elements = result.find_all("li")

        for li_element in li_elements:
            try:
                li_element.span.unwrap()
            except:
                pass
            try:
                if li_element.getText()[-1] != '.' and li_element.getText()[-1] != ':' and li_element.getText()[-1] != '?':
                    #li_element.insert(1,NavigableString("."))
                    li_element.append(NavigableString("."))
            except:
                pass

                
        text = result.getText(separator=u' ')

        result2 = soup2.find("div", class_="SP-ArticleContent")

        for div in result2.find_all("section", class_="SP-Section SP-Grid__full"):
            for h2 in div.find_all("h2"):
                if(h2.text == 'Ihr Kontakt zu uns:'):
                    div.decompose()

        for div in result2.find_all("section", class_="SP-Text"):
            for h2 in div.find_all("h2", id="service-telefon"):
                div.decompose()    

        text2 = result2.getText(separator=u' ')    

        df_article = pd.DataFrame({
            'URL': url,
            'Titel': title,
            'Text_Leicht': text,
            'Text_Allgemein': text2},
            index=[i]
        )

        i += 1
        df = pd.concat([df, df_article])
    except Exception as error:
        print("Error with ", title, url)
        print(error)

Personalausweis beantragen - erstmalig oder nach Ablauf https://stuttgart.de/organigramm/leistungen/personalausweis-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy
Personalausweis wegen Namenänderung beantragen (Heirat oder Scheidung) https://stuttgart.de/organigramm/leistungen/personalausweis-wegen-namenaenderung-beantragen-heirat-oder-scheidung-.php?sp%3Aout=easy
Reisepass beantragen - erstmalig oder nach Ablauf https://stuttgart.de/organigramm/leistungen/reisepass-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy
Reisepass beantragen - vorläufig https://stuttgart.de/organigramm/leistungen/reisepass-beantragen-vorlaeufig.php?sp%3Aout=easy
Schwerbehinderten-Ausweis beantragen https://stuttgart.de/organigramm/leistungen/schwerbehinderten-ausweis-beantragen.php?sp%3Aout=easy
Urkunden und beglaubigte Abschriften: Geburt, Heirat, Lebens-Partnerschaft und Sterbefall https://stuttgart.de/organigramm/leistungen/urkunden-und-beglaubigte-abschriften-geburt-heirat-lebens-partnersc

In [23]:
df.to_csv("../../02_Data/stuttgart_4.csv")

In [113]:
#https://stuttgart.de/organigramm/leistungen/personalausweis-wegen-namenaenderung-beantragen-heirat-oder-scheidung-.php?sp%3Aout=easy
url = 'https://www.stuttgart.de/organigramm/leistungen/personalausweis-beantragen-erstmalig-oder-nach-ablauf.php?sp%3Aout=easy'
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")
result = soup.find("div", class_="SP-ArticleContent")


In [114]:
from bs4 import NavigableString

li_elements = result.find_all("li")

for li_element in li_elements:
    try:
        li_element.span.unwrap()
    except:
        pass

    if li_element.getText()[-1] != '.' and li_element.getText()[-1] != ':' and li_element.getText()[-1] != '?':
        li_element.insert(1,NavigableString("."))
   
    print(li_element.getText())


<li>die Entsperr-Nummer.<br/></li>
die Entsperr-Nummer.
<li>das Sperr-Kennwort<br/>  </li>
das Sperr-Kennwort.  
<li>weitere Informationen über das Sperren von der eID-Funktion.</li>
weitere Informationen über das Sperren von der eID-Funktion.
<li>Sie beantragen den Personal-Ausweis bei der Auslands-Vertretung in Ihrem Bezirk.<br/></li>
Sie beantragen den Personal-Ausweis bei der Auslands-Vertretung in Ihrem Bezirk.
<li>Sie können auch in Deutschland den Personal-Ausweis beantragen.<br/></li>
Sie können auch in Deutschland den Personal-Ausweis beantragen.
<li>Sie beantragen den Personal-Ausweis zum ersten Mal?<br/></li>
Sie beantragen den Personal-Ausweis zum ersten Mal?
<li>Sie haben <strong>keinen</strong> Ausweis und <strong>keinen</strong> Pass?<br/></li>
Sie haben keinen Ausweis und keinen Pass?
<li>Sie beantragen einen neuen Personal-Ausweis?<br/></li>
Sie beantragen einen neuen Personal-Ausweis?
<li><span><span> </span></span><span>Ein Foto von Ihnen<br/></span></li>
 .Ein Foto 

In [50]:
#https://www.stadt-koeln.de/leben-in-koeln/soziales/hund-anmelden

#https://stuttgart.de/organigramm/leistungen/personalausweis-wegen-namenaenderung-beantragen-heirat-oder-scheidung-.php?sp%3Aout=easy
url = 'https://www.stadt-koeln.de/leben-in-koeln/soziales/hund-anmelden'

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

li_elements = soup.find_all("li")

for li_element in li_elements:
    #print(li_element.prettify(), end="\n"*2)
    print('####')
    print(li_element.getText())

####
Website Translation Widget by conword.io
####
Website Translation Widget by conword.io
####

Service

####

Leben in Köln
Zur Übersichtsseite Leben in Köln



Bildung und Schule
Bildungspaket

Digitale Bildung

Angebote des Schulpsychologischen Dienstes für Lehrkräfte

Ganztagsangebote

Inklusion und Förderung

Rheinische Musikschule

Schulformen

Stadtbibliothek

Volkshochschule




Kultur
Historisches Archiv

Kölner Kulturorte

Kulturförderung

Opernquartier

Provenienzforschung

Rheinisches Bildarchiv






Freizeit, Natur und Sport
Betriebssportgemeinschaft

Ferien und Freizeit

Friedhöfe

Karneval

Kölner Wald und Grün

Parks und Gärten

Planung und Projekte

Sportstadt Köln

Veranstaltungskalender




Planen und Bauen
Bebauungspläne

Denkmalschutz und Denkmalpflege

Grundstücks- und Immobilienservice

Karten, Kataster und Vermessung




Sicherheit und Ordnung




Familie, Partnerschaft und Kinder
Ehe und Lebenspartnerschaft

Jugendschutz

Kinder- und Jugendpädagogische Einri